### Lab 2- Facebook AI Similarity Search (FAISS) Index Partitioning

Following up with Lab 1, when using large data sets, flat index and exhaustive sarch are not ideal for performing similarity search. This would require paritioning the index for performing efficient search. For paritioning let's introduce the concept of voronai diagrams. In mathematics, a Voronoi diagram is a partition of a plane into regions close to each of a given set of objects. A good example is let's take the country of Canada. Canada has many provinces. In a Voronai diagram, Canada can be split into 4 groups of provinces. Atlantic Canada, Eastern provinces, Prairies and Western provinces. Provinces Nova scotia, New Brunswick, New foundland labrador and Prince Edward Island. A Tourist visiting a procince insite Atlantic Canada has more probability to visit the other  provinces within Atlantic Canada than in the Prairies or western canada. The same principle applies to vector search considering equal distances between the results.

Ithe previous lab, we used IndexFlatL2 index to build and store vectors. In this lab, you will use voronai cells to parition the index to optimize the performance of search results. 

In [ ]:
# 
# 1. This lab can requires some large datasets 
# 2. You will split document into sentences
# 3. Create a new index and train it on the data
# 4. Split the index into partitions of voronai cells
# 5. Given a query, i.e. "Who plays foot ", you find the K most similar sentences
# 6. Adjust the "k" parameter to explore speed vs accuracy vs approximation

In [ ]:
# You will the need python libraries for this tutorial. A basic understanding of python is required. 
# You can install the libraries using pip if not in your notebook pre-installed. 

In [ ]:
!pip install faiss-cpu
import requests
from io import StringIO
import pandas as pd
import numpy as np
import faiss

In [ ]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t', on_bad_lines='skip')
data.head()

In [ ]:
# You take all the sentences from into a python list 
# You will get an output of 4.5K sentences
sentences = data['sentence_A'].tolist()
sentences[:5]
len(sentences)

In [ ]:
# You take all samples from both sentence A and B and merge them together
# You will get ~4.8K unique sentences
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)   
len(set(sentences))  

In [ ]:
# Still the dataset is small. You are going to add more data by parsing the data from below URLS
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/headlines.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/belief.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/answers-students.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/answers-forums.test.tsv'
]

In [ ]:
# Each of these datasets have the same structure, so we loop through each creating our sentences data
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data = pd.read_csv(StringIO(res.text), sep='\t', header=None, on_bad_lines='skip')
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

In [ ]:
# Let's clean up the data by removin duplicates and NaN 
# You will get approximately 25k sentences
sentences = [word for word in list(set(sentences)) if type(word) is str]
len(sentences)

In [ ]:
# You need to install sentence_transformers library. This framework provides an easy method to compute 
# dense vector representations for sentences, paragraphs, and images.
# For additional reading https://pypi.org/project/sentence-transformers/
!pip install sentence-transformers

In [ ]:
# The models are based on transformer networks like BERT / RoBERTa / XLM-RoBERTa etc. 
# and achieve state-of-the-art performance in various task. Read the pypi library link about supported models. 
# You need to initialize sentence transformer model. 
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
# create sentence embeddings using the multi-qa-MiniLM-L6 model from hugging face
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

In [ ]:
# Let's get the dimensions 
d = sentence_embeddings.shape[1]
d

In [ ]:
# IndexIVFFlat - Inverted File index. Inverted file index takes two parameters
# nlist : The number of clusters to be formed. These clusters are the voronai cells
# quantizer : to assign the vectors to a particular cluster. This is usually another index that uses the L2 Euclidian distance metric (we use the IndexFlatL2 index)
nlist = 50
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

In [ ]:
# Check to see if the index is trained. In the previous IndexFlatL2 training was not required and it will return true
# When using voronai cells in Inverted file index, training the cells is required. The function below will return false.
index.is_trained

In [ ]:
# Let's train the index and check if index is now trained. It should return "True"
index.train(sentence_embeddings)
index.is_trained  

In [ ]:
# Let's add the vectors in to the index
index.add(sentence_embeddings)

In [ ]:
index.ntotal

In [ ]:
%%time
# Now that our index is trained, let's query the index
# "xq" Query vector
# "nprobe" parameter specifies the number of clusters to visit during the search operation
# "k" specifies the number of similar vectors to be returned from the visited clusters.
#Then search with a given query `xq` and number of nearest neigbors to return `k`.
index.nprobe=2
k = 4
xq = model.encode(["Who is playing football"])
D, I = index.search(xq, k)  # search
print(I)

In [ ]:
#You will be get 4 nearest locations returned by the query. Along with this you will know how long it takes to return the results.

In [ ]:
# Let's see the results of query and 4 nearest neighbours related to Jeff Bezos and Internet
for i,location in enumerate(I[0].tolist()):
    print(location, ":", sentences[location])

In [ ]:
%%time
# "nprobe" parameter specifies the number of clusters to visit during the search operation
# Let's increase the scope of clusters to search 
index.nprobe=4
k = 4
xq = model.encode(["Who is playing football"])
D, I = index.search(xq, k)  # search
print(I)

In [ ]:
%%time
# "nprobe" parameter specifies the number of clusters to visit during the search operation
# Let's increase the scope of clusters to search 
index.nprobe=8
k = 4
xq = model.encode(["Who is playing football"])
D, I = index.search(xq, k)  # search
print(I)

In [ ]:
%%time
# "nprobe" parameter specifies the number of clusters to visit during the search operation
# Let's increase the scope of clusters to search 
index.nprobe=16
k = 4
xq = model.encode(["Who is playing football"])
D, I = index.search(xq, k)  # search
print(I)

In [ ]:
# We are searching large number of clusters in order of 2, 4,8 and 16 and still receive responses faster than IndexFlatL2-only index 
# Is this the final step of optimization and can we optimize the performance further
# Next Step: Product quantization 